In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
database_path="../Datasets/database/longlist.db"
conn = sqlite3.connect(database_path)
query_path="Queries/longlist/"

# Create a cursor object
cursor = conn.cursor()

# Longlist

```mermaid
erDiagram
  authors {
    INTEGER id PK
    TEXT name
    TEXT country
    INTEGER birth
  }
  authored {
    INTEGER author_id
    INTEGER book_id
  }
  authored ||--o{ books : "book_id → id"
  authored ||--o{ authors : "author_id → id"
  books {
    INTEGER id PK
    TEXT isbn
    TEXT title
    INTEGER publisher_id
    TEXT format
    INTEGER pages
    TEXT published
    INTEGER year
  }
  books ||--o{ publishers : "publisher_id → id"
  publishers {
    INTEGER id PK
    TEXT publisher
  }
  ratings {
    INTEGER book_id
    INTEGER rating
  }
  ratings ||--o{ books : "book_id → id"
  translators {
    INTEGER id PK
    TEXT name
  }
  translated {
    INTEGER translator_id
    INTEGER book_id
  }
  translated ||--o{ books : "book_id → id"
  translated ||--o{ translators : "translator_id → id"
  ```

In [69]:
# # Get database info: List all tables
# tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
# tables = pd.read_sql(tables_query, conn)
# print("Database Tables:\n", tables)

In [70]:
# Get column info for the 'episodes' table
table="authors"
columns_query = f"PRAGMA table_info({table});"
columns = pd.read_sql(columns_query, conn)
print("\nTable Schema for 'episodes':\n", columns)


Table Schema for 'episodes':
    cid     name     type  notnull dflt_value  pk
0    0       id  INTEGER        0       None   1
1    1     name     TEXT        0       None   0
2    2  country     TEXT        0       None   0
3    3    birth  INTEGER        0       None   0


In [71]:
### Loading from SQL file
# load from a file
query="load_test.sql"
qpath=query_path+query

# load query
with open(qpath, "r") as file:
    query=file.read()
# execute query
df = pd.read_sql(query, conn)
print("---", df)
conn.close()

---                    name
0         Adania Shibli
1         Ahmed Saadawi
2   Alia Trabucco Zerán
3       Amanda Svensson
4         Andrey Kurkov
..                  ...
67   Virginie Despentes
68         Willem Anker
69           Wu Ming-Yi
70           Yōko Ogawa
71          Zou Jingzhi

[72 rows x 1 columns]


### General

In [3]:
# Connect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

In [73]:
# Query 1: Select all publishers
df_publishers = pd.read_sql("SELECT * FROM publishers LIMIT 5", conn)
print("\nPublishers:\n", df_publishers)


Publishers:
    id              publisher
0   1      And Other Stories
1   2           Charco Press
2   3        Europa Editions
3   4          Faber & Faber
4   5  Fitzcarraldo Editions


In [74]:
# Query 2: Find ID of 'MacLehose Press'
df_publisher_id = pd.read_sql("SELECT id FROM publishers WHERE publisher = 'MacLehose Press'", conn)
print("\nMacLehose Press ID:\n", df_publisher_id)


MacLehose Press ID:
    id
0  12


In [75]:
# Query 3: Find books published by MacLehose Press
df_books = pd.read_sql("SELECT title FROM books WHERE publisher_id = 12", conn)
print("\nBooks by MacLehose Press:\n", df_books)


Books by MacLehose Press:
                        title
0             Standing Heavy
1  Jimi Hendrix Live in Lviv
2     An Inventory of Losses
3      The Faculty of Dreams
4     The Shape of the Ruins
5               The Impostor
6           Vernon Subutex 1


In [ ]:
# Query 4: Find books by nested query
df_books_nested = pd.read_sql("""
    SELECT title FROM books WHERE publisher_id = (
        SELECT id FROM publishers WHERE publisher = 'MacLehose Press'
    )""", conn)
print("\nBooks by MacLehose Press (Nested Query):\n", df_books_nested)


Books by MacLehose Press (Nested Query):
                        title
0             Standing Heavy
1  Jimi Hendrix Live in Lviv
2     An Inventory of Losses
3      The Faculty of Dreams
4     The Shape of the Ruins
5               The Impostor
6           Vernon Subutex 1


In [ ]:
# # Query 5: Find ratings for 'In Memory of Memory'
# df_ratings = pd.read_sql("""
#     SELECT rating FROM ratings WHERE book_id = (
#         SELECT id FROM books WHERE title = 'In Memory of Memory'
#     )""", conn)
# print("\nRatings for 'In Memory of Memory':\n", df_ratings)

# Query 6: Find the average rating for 'In Memory of Memory'
df_avg_rating = pd.read_sql("""
    SELECT AVG(rating) FROM ratings WHERE book_id = (
        SELECT id FROM books WHERE title = 'In Memory of Memory'
    )""", conn)
print("\nAverage Rating for 'In Memory of Memory':\n", df_avg_rating)


Average Rating for 'In Memory of Memory':
    AVG(rating)
0      3.86036


In [4]:
# one way then the other

# Query 7: Find author of 'Flights'
df_author = pd.read_sql("""
    SELECT name FROM authors WHERE id = (
        SELECT author_id FROM authored WHERE book_id = (
            SELECT id FROM books WHERE title = 'Flights'
        )
    )""", conn)
print("\nAuthor of 'Flights':\n", df_author)

# # Query 8: Find books by 'Fernanda Melchor'
# df_books_melchor = pd.read_sql("""
#     SELECT title FROM books WHERE id IN (
#         SELECT book_id FROM authored WHERE author_id = (
#             SELECT id FROM authors WHERE name = 'Fernanda Melchor'
#         )
#     )""", conn)
# print("\nBooks by Fernanda Melchor:\n", df_books_melchor)


Author of 'Flights':
              name
0  Olga Tokarczuk


In [ ]:
# Query 9: Find books with 'love' in the title
df_love_books = pd.read_sql("SELECT title FROM books WHERE title LIKE '%love%'", conn)
print("\nBooks with 'love' in title:\n", df_love_books)

# # Query 10: Find books starting with 'The'
# df_the_books = pd.read_sql("SELECT title FROM books WHERE title LIKE 'The %'", conn)
# print("\nBooks starting with 'The':\n", df_the_books)


Books with 'love' in title:
                         title
0        Love in the Big City
1    More Than I Love My Life
2  Love in the New Millennium
3                Die, My Love


In [ ]:
# Query 11: Find books matching uncertain title spelling
df_uncertain_book = pd.read_sql("SELECT title FROM books WHERE title LIKE 'P_re'", conn)
print("\nBooks matching uncertain spelling:\n", df_uncertain_book)



Books matching uncertain spelling:
   title
0  Pyre


In [5]:
# # Query 12: Find average rating of all books
# df_avg_all = pd.read_sql("SELECT ROUND(AVG(rating), 2) AS 'Average Rating' FROM ratings", conn)
# print("\nAverage rating of all books:\n", df_avg_all)

# Get the average rating for book authored by Fernanda Melchor
df_avg_melchor = pd.read_sql("""
    SELECT ROUND(AVG(rating), 2) AS 'Average Rating' FROM ratings WHERE book_id IN (
        SELECT book_id FROM authored WHERE author_id = (
            SELECT id FROM authors WHERE name = 'Fernanda Melchor'
        )
    )""", conn)
print("\nAverage rating for books by Fernanda Melchor:\n", df_avg_melchor)



Average rating for books by Fernanda Melchor:
    Average Rating
0            3.98


In [ ]:
# Query 13: Find maximum and minimum rating
df_max_rating = pd.read_sql("SELECT MAX(rating) FROM ratings", conn)
df_min_rating = pd.read_sql("SELECT MIN(rating) FROM ratings", conn)
print("\nMaximum rating:\n", df_max_rating)
print("\nMinimum rating:\n", df_min_rating)



Maximum rating:
    MAX(rating)
0            5

Minimum rating:
    MIN(rating)
0            1


In [ ]:
# Query 14: Count total votes (pages)
df_total_pages = pd.read_sql("SELECT SUM(pages) FROM books", conn)
print("\nTotal number of pages:\n", df_total_pages)



Total number of pages:
    SUM(pages)
0       22345


In [6]:
# Query 15: Count total books and translators
df_total_books = pd.read_sql("SELECT COUNT(*) FROM books", conn)
df_total_translators = pd.read_sql("SELECT COUNT(name) FROM translators", conn)
print("\nTotal number of books:\n", df_total_books)
print("\nTotal number of translators:\n", df_total_translators)



Total number of books:
    COUNT(*)
0        78

Total number of translators:
    COUNT(name)
0           74


In [8]:
# Query 16: Count unique publishers
df_unique_publishers = pd.read_sql("SELECT COUNT(DISTINCT publisher) FROM publishers", conn)
print("\nTotal unique publishers:\n", df_unique_publishers)


Total unique publishers:
    COUNT(DISTINCT publisher)
0                         33


In [9]:
# Close connection
conn.close()


### Sets

In [11]:
# Connect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Query 1: UNION - Combines authors and translators into one result set
df_union = pd.read_sql("""
    SELECT 'author' AS profession, name FROM authors
    UNION
    SELECT 'translator' AS profession, name FROM translators;
""", conn)
print("\nAuthors and Translators:\n", df_union)



Authors and Translators:
      profession                 name
0        author        Adania Shibli
1        author        Ahmed Saadawi
2        author  Alia Trabucco Zerán
3        author      Amanda Svensson
4        author        Andrey Kurkov
..          ...                  ...
141  translator         Sophie Lewis
142  translator     Sora Kim-Russell
143  translator       Stephen Snyder
144  translator      Susan Bernofsky
145  translator         Tiffany Tsao

[146 rows x 2 columns]


In [ ]:
# Query 2: INTERSECT - Finds authors who are also translators
df_intersect = pd.read_sql("""
    SELECT name FROM authors
    INTERSECT
    SELECT name FROM translators;
""", conn)
print("\nAuthors who are also Translators:\n", df_intersect)



Authors who are also Translators:
                 name
0  Ngũgĩ wa Thiong'o


In [12]:
# Query 3: Finds books translated by Sophie Hughes
df_sophie_books = pd.read_sql("""
    SELECT book_id FROM translated WHERE translator_id = (
        SELECT id FROM translators WHERE name = 'Sophie Hughes'
    );
""", conn)
print("\nBooks translated by Sophie Hughes:\n", df_sophie_books)



Books translated by Sophie Hughes:
    book_id
0       14
1       48
2       50
3       63


In [13]:
# Query 3: Finds books translated by Sophie Hughes
df_margaret_books = pd.read_sql("""
    SELECT book_id FROM translated WHERE translator_id = (
        SELECT id FROM translators WHERE name = 'Margaret Jull Costa'
    );
""", conn)
print("\nBooks translated by Sophie Hughes:\n", df_margaret_books)



Books translated by Sophie Hughes:
    book_id
0       50


In [ ]:
# Query 5: INTERSECT - Finds books translated by both Sophie Hughes and Margaret Jull Costa
df_common_books = pd.read_sql("""
    SELECT book_id FROM translated WHERE translator_id = (
        SELECT id FROM translators WHERE name = 'Sophie Hughes'
    )
    INTERSECT
    SELECT book_id FROM translated WHERE translator_id = (
        SELECT id FROM translators WHERE name = 'Margaret Jull Costa'
    );
""", conn)
print("\nBooks translated by both Sophie Hughes and Margaret Jull Costa:\n", df_common_books)



Books translated by both Sophie Hughes and Margaret Jull Costa:
    book_id
0       50


In [ ]:
# Query 6: Retrieves the titles of books translated by both translators
df_common_titles = pd.read_sql("""
    SELECT title FROM books WHERE id IN (
        SELECT book_id FROM translated WHERE translator_id = (
            SELECT id FROM translators WHERE name = 'Sophie Hughes'
        )
        INTERSECT
        SELECT book_id FROM translated WHERE translator_id = (
            SELECT id FROM translators WHERE name = 'Margaret Jull Costa'
        )
    );
""", conn)
print("\nTitles of books translated by both Sophie Hughes and Margaret Jull Costa:\n", df_common_titles)



Titles of books translated by both Sophie Hughes and Margaret Jull Costa:
                  title
0  Mac and His Problem


In [97]:

# Query 7: EXCEPT - Finds translators who are not authors
df_translators_not_authors = pd.read_sql("""
    SELECT name FROM translators
    EXCEPT
    SELECT name FROM authors;
""", conn)
print("\nTranslators who are not Authors:\n", df_translators_not_authors)



Translators who are not Authors:
                     name
0     Adrian Nathan West
1      Alison L. Strayer
2           Angela Rodel
3   Aniruddhan Vasudevan
4       Anna Moschovakis
..                   ...
68          Sophie Lewis
69      Sora Kim-Russell
70        Stephen Snyder
71       Susan Bernofsky
72          Tiffany Tsao

[73 rows x 1 columns]


In [98]:

# Close the connection
conn.close()


### Groups

In [99]:
# Connect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Query 1: Finds average rating for each book
df_avg_ratings = pd.read_sql("""
    SELECT book_id, ROUND(AVG(rating), 2) AS "average rating" FROM ratings
    GROUP BY book_id;
""", conn)
print("\nAverage rating for each book:\n", df_avg_ratings)



Average rating for each book:
     book_id  average rating
0         1            3.77
1         2            3.97
2         3            3.04
3         4            3.57
4         5            4.06
..      ...             ...
73       74            3.83
74       75            3.77
75       76            3.89
76       77            3.54
77       78            3.76

[78 rows x 2 columns]


In [ ]:
# Query 2: Joins titles with average ratings
df_avg_ratings_with_titles = pd.read_sql("""
    SELECT title, ROUND(AVG(rating), 2) AS "average rating" FROM ratings
    JOIN books ON books.id = ratings.book_id
    GROUP BY book_id;
""", conn)
print("\nAverage rating per book title:\n", df_avg_ratings_with_titles)



Average rating per book title:
                                     title  average rating
0                                 Boulder            3.77
1                                   Whale            3.97
2   The Gospel According to the New World            3.04
3                          Standing Heavy            3.57
4                            Time Shelter            4.06
..                                    ...             ...
73                         The White Book            3.83
74                      The World Goes On            3.77
75                       Vernon Subutex 1            3.89
76                           Die, My Love            3.54
77                                Flights            3.76

[78 rows x 2 columns]


In [ ]:
# Query 3: Chooses books with a rating of 4.0 or higher
df_high_rated_books = pd.read_sql("""
    SELECT title, ROUND(AVG(rating), 2) AS "average rating" FROM ratings
    JOIN books ON books.id = ratings.book_id
    GROUP BY book_id
    HAVING "average rating" > 4.0;
""", conn)
print("\nBooks with an average rating above 4.0:\n", df_high_rated_books)



Books with an average rating above 4.0:
                                     title  average rating
0                            Time Shelter            4.06
1                                    Pyre            4.04
2                              Still Born            4.14
3                             Elena Knows            4.09
4                More Than I Love My Life            4.04
5            A New Name: Septology VI-VII            4.50
6                      The Books of Jacob            4.05
7   When We Cease to Understand the World            4.14
8                         The Eighth Life            4.51
9          The Other Name: Septology I-II            4.19
10                       Hurricane Season            4.08
11                              The Years            4.18
12                    The Flying Mountain            4.11


In [102]:

# Query 4: Chooses books with a rating of 4.0 or higher, ordered by rating
df_high_rated_books_ordered = pd.read_sql("""
    SELECT book_id, ROUND(AVG(rating), 2) AS "average rating" FROM ratings
    GROUP BY book_id
    HAVING "average rating" > 4.0
    ORDER BY "average rating" DESC;
""", conn)
print("\nBooks with an average rating above 4.0, ordered by rating:\n", df_high_rated_books_ordered)



Books with an average rating above 4.0, ordered by rating:
     book_id  average rating
0        42            4.51
1        22            4.50
2        45            4.19
3        65            4.18
4        28            4.14
5        11            4.14
6        71            4.11
7        18            4.09
8        48            4.08
9         5            4.06
10       25            4.05
11       20            4.04
12       10            4.04


In [103]:

# Close the connection
conn.close()

### Nested

In [104]:
# Connect to the database
conn = sqlite3.connect(database_path)
cursor = conn.cursor()

# Query 1: Finds all books published by MacLehose Press
df_maclehose_books = pd.read_sql("""
    SELECT title FROM books WHERE publisher_id = (
        SELECT id FROM publishers WHERE publisher = 'MacLehose Press'
    );
""", conn)
print("\nBooks published by MacLehose Press:\n", df_maclehose_books)



Books published by MacLehose Press:
                        title
0             Standing Heavy
1  Jimi Hendrix Live in Lviv
2     An Inventory of Losses
3      The Faculty of Dreams
4     The Shape of the Ruins
5               The Impostor
6           Vernon Subutex 1


In [105]:

# Query 2: Finds all ratings for "In Memory of Memory"
df_memory_ratings = pd.read_sql("""
    SELECT rating FROM ratings WHERE book_id = (
        SELECT id FROM books WHERE title = 'In Memory of Memory'
    );
""", conn)
print("\nRatings for 'In Memory of Memory':\n", df_memory_ratings)



Ratings for 'In Memory of Memory':
       rating
0          3
1          4
2          4
3          4
4          3
...      ...
1549       4
1550       4
1551       4
1552       4
1553       4

[1554 rows x 1 columns]


In [106]:

# Query 3: Finds average rating for "In Memory of Memory"
df_memory_avg_rating = pd.read_sql("""
    SELECT AVG(rating) FROM ratings WHERE book_id = (
        SELECT id FROM books WHERE title = 'In Memory of Memory'
    );
""", conn)
print("\nAverage rating for 'In Memory of Memory':\n", df_memory_avg_rating)



Average rating for 'In Memory of Memory':
    AVG(rating)
0      3.86036


In [107]:

# Query 4: Finds author who wrote "The Birthday Party"
df_birthday_author = pd.read_sql("""
    SELECT name FROM authors WHERE id = (
        SELECT author_id FROM authored WHERE book_id = (
            SELECT id FROM books WHERE title = 'The Birthday Party'
        )
    );
""", conn)
print("\nAuthor of 'The Birthday Party':\n", df_birthday_author)



Author of 'The Birthday Party':
                  name
0  Laurent Mauvignier


In [108]:

# Query 5: Finds all books by Fernanda Melchor
df_melchor_books = pd.read_sql("""
    SELECT title FROM books WHERE id IN (
        SELECT book_id FROM authored WHERE author_id = (
            SELECT id FROM authors WHERE name = 'Fernanda Melchor'
        )
    );
""", conn)
print("\nBooks by Fernanda Melchor:\n", df_melchor_books)



Books by Fernanda Melchor:
               title
0          Paradais
1  Hurricane Season


In [109]:

# Query 6: Finds books by multiple authors (Fernanda Melchor, Annie Ernaux)
df_multiple_authors_books = pd.read_sql("""
    SELECT title FROM books WHERE id IN (
        SELECT book_id FROM authored WHERE author_id IN (
            SELECT id FROM authors WHERE name IN ('Fernanda Melchor', 'Annie Ernaux')
        )
    );
""", conn)
print("\nBooks by Fernanda Melchor or Annie Ernaux:\n", df_multiple_authors_books)



Books by Fernanda Melchor or Annie Ernaux:
               title
0          Paradais
1  Hurricane Season
2         The Years


In [110]:

# Close the connection
conn.close()


# Sea Lions

```mermaid
erDiagram
  sea_lions {
    INTEGER id PK
    TEXT name
    TEXT species
  }
  migrations {
    INTEGER id PK
    INTEGER distance
    INTEGER days
  }
```

In [50]:
# Connect to the database
database_path="../Datasets/database/sea_lions.db"
conn = sqlite3.connect(database_path)

# Query 1: Join two tables using primary keys
df_join = pd.read_sql("SELECT * FROM sea_lions JOIN migrations ON migrations.id = sea_lions.id", conn)
print("\nJoined sea_lions and migrations:\n", df_join)


Joined sea_lions and migrations:
       id   name                 species     id  distance  days
0  10484   Ayah  Zalophus californianus  10484      1000   107
1  11728   Spot  Zalophus californianus  11728      1531    56
2  11729  Tiger  Zalophus californianus  11729      1370    37
3  11732  Mabel  Zalophus californianus  11732      1622    62
4  11734   Rick  Zalophus californianus  11734      1491    58


In [51]:
# Query 2: Left join
df_left_join = pd.read_sql("SELECT * FROM sea_lions LEFT JOIN migrations ON migrations.id = sea_lions.id", conn)
print("\nLeft Join Result:\n", df_left_join)



Left Join Result:
       id   name                 species       id  distance   days
0  10484   Ayah  Zalophus californianus  10484.0    1000.0  107.0
1  11728   Spot  Zalophus californianus  11728.0    1531.0   56.0
2  11729  Tiger  Zalophus californianus  11729.0    1370.0   37.0
3  11732  Mabel  Zalophus californianus  11732.0    1622.0   62.0
4  11734   Rick  Zalophus californianus  11734.0    1491.0   58.0
5  11790  Jolee  Zalophus californianus      NaN       NaN    NaN


In [52]:
# Query 3: Right join (SQLite does not support RIGHT JOIN directly)
df_right_join = pd.read_sql("SELECT * FROM migrations LEFT JOIN sea_lions ON sea_lions.id = migrations.id", conn)
print("\nRight Join Result:\n", df_right_join)


Right Join Result:
       id  distance  days       id   name                 species
0  10484      1000   107  10484.0   Ayah  Zalophus californianus
1  11728      1531    56  11728.0   Spot  Zalophus californianus
2  11729      1370    37  11729.0  Tiger  Zalophus californianus
3  11732      1622    62  11732.0  Mabel  Zalophus californianus
4  11734      1491    58  11734.0   Rick  Zalophus californianus
5  11735      2723    82      NaN   None                    None
6  11736      1571    52      NaN   None                    None
7  11737      1957    92      NaN   None                    None


In [53]:
# Query 4: Full join (Simulated using UNION in SQLite)
df_full_join = pd.read_sql("""
SELECT * FROM sea_lions LEFT JOIN migrations ON migrations.id = sea_lions.id
UNION
SELECT * FROM sea_lions RIGHT JOIN migrations ON migrations.id = sea_lions.id
""", conn)
print("\nFull Join Result:\n", df_full_join)


Full Join Result:
         id   name                 species       id  distance   days
0      NaN   None                    None  11735.0    2723.0   82.0
1      NaN   None                    None  11736.0    1571.0   52.0
2      NaN   None                    None  11737.0    1957.0   92.0
3  10484.0   Ayah  Zalophus californianus  10484.0    1000.0  107.0
4  11728.0   Spot  Zalophus californianus  11728.0    1531.0   56.0
5  11729.0  Tiger  Zalophus californianus  11729.0    1370.0   37.0
6  11732.0  Mabel  Zalophus californianus  11732.0    1622.0   62.0
7  11734.0   Rick  Zalophus californianus  11734.0    1491.0   58.0
8  11790.0  Jolee  Zalophus californianus      NaN       NaN    NaN


In [54]:
# df=pd.read_sql("SELECT * FROM sea_lions", conn)

In [55]:
# Query 5: Natural join
df_natural_join = pd.read_sql("SELECT * FROM sea_lions NATURAL JOIN migrations", conn)
print("\nNatural Join Result:\n", df_natural_join)


Natural Join Result:
       id   name                 species  distance  days
0  10484   Ayah  Zalophus californianus      1000   107
1  11728   Spot  Zalophus californianus      1531    56
2  11729  Tiger  Zalophus californianus      1370    37
3  11732  Mabel  Zalophus californianus      1622    62
4  11734   Rick  Zalophus californianus      1491    58


In [56]:
# Query 6: Join with WHERE condition
df_filtered = pd.read_sql("SELECT * FROM sea_lions JOIN migrations ON migrations.id = sea_lions.id WHERE migrations.distance > 1500", conn)
print("\nSea Lions with Migration Distance > 1500:\n", df_filtered)


Sea Lions with Migration Distance > 1500:
       id   name                 species     id  distance  days
0  11728   Spot  Zalophus californianus  11728      1531    56
1  11732  Mabel  Zalophus californianus  11732      1622    62


In [57]:

# Close connection
conn.close()